
# Milvus GPU on WSL2 — Complete Setup Guide

This notebook provides a **clean, sequential setup** for running Milvus with GPU support on WSL2 using Docker Desktop.

## Before you start

Make sure the Windows host already has the following pieces in place:

| Requirement | Why it matters | How to check |
| --- | --- | --- |
| Windows 11/10 with WSL2 enabled | WSL2 provides the Linux environment Milvus runs in. | Run `wsl --status` in PowerShell. |
| Ubuntu 22.04 or 24.04 distro for WSL2 | Commands below assume an Ubuntu base image with systemd. | `wsl -l -v` shows distro name/version. |
| Docker Desktop (WSL2 integration enabled) | Provides Docker engine + compose for launching Milvus. | Docker Desktop → Settings → Resources → WSL Integration. |
| NVIDIA GPU with latest Windows driver | The Windows driver exposes GPU to WSL and containers. | `nvidia-smi` from a regular Command Prompt. |
| CUDA-enabled WSL2 support packages | Required for GPU passthrough to Docker containers. | `nvidia-smi` inside WSL after Step 5. |

If you are missing any prerequisite, address it first before moving on—this guide assumes those foundations exist.



## Step 0 — Confirm baseline (WSL2, systemd, Docker Desktop integration)

From your Ubuntu WSL shell:

```bash
# Confirm WSL2 kernel
uname -r
cat /proc/version

# Check if systemd is running (Ubuntu 22.04/24.04 on WSL2 usually has it)
systemctl is-system-running || true

# Ensure Docker CLI can reach daemon (daemon runs inside docker-desktop)
docker version
docker info | head -n 20
```

If `docker version` fails, open **Docker Desktop → Settings → Resources → WSL Integration** and check your Ubuntu distro.



## Step 1 — Prereqs (Ubuntu WSL)

```bash
sudo apt-get update
sudo apt-get install -y curl gnupg ca-certificates
```



## Step 2 — Install NVIDIA repo GPG key (with `signed-by` keyring)

These commands teach `apt` to trust the official NVIDIA container repository and define where its key lives:

1. **Create a shared keyring folder** so multiple NVIDIA lists can reference the same key.
2. **Download and convert the GPG key** into `/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg`.
3. **Relax permissions** so the `apt` process can read the keyring.
4. **Install the repo list** that points to NVIDIA packages and tells `apt` to use the new keyring via `signed-by`.

```bash
sudo install -m 0755 -d /usr/share/keyrings

curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey \
  | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg

sudo chmod 644 /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg

curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list \
  | sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' \
  | sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list >/dev/null
```



## Step 3 — Install NVIDIA Container Toolkit

```bash
sudo apt-get update
sudo apt-get install -y nvidia-container-toolkit
```



## Step 4 — Configure Docker runtime & restart Docker Desktop

```bash
# Write default-runtime=nvidia into /etc/docker/daemon.json (Ubuntu side)
sudo nvidia-ctk runtime configure --runtime=docker --set-as-default

# Restart Docker Desktop daemon (lives in docker-desktop distro):
# Option A: Right-click Docker Desktop icon → Restart
# Option B: CLI — shut down all distros and re-open Docker Desktop
wsl --shutdown
```

> On Docker Desktop, `docker info` may still show `Default Runtime: runc`.  
> If you want `nvidia` to be global default, open **Docker Desktop → Settings → Docker Engine** and merge:
> ```json
> {
>   "default-runtime": "nvidia",
>   "runtimes": {
>     "nvidia": { "path": "nvidia-container-runtime", "args": [] }
>   }
> }
> ```
> then click **Apply & Restart**.



## Step 5 — Verify GPU access inside container

Run CUDA base image and check `nvidia-smi`:

```bash
docker run --rm --gpus all nvidia/cuda:12.5.1-base-ubuntu24.04 nvidia-smi
```

Expected: RTX GPU details, driver version, CUDA version, and `No running processes found`.



## Step 6 — Deploy Milvus Standalone (GPU) via official compose file

Create working folder and download the GPU compose file:

```bash
mkdir -p ~/milvus && cd ~/milvus
wget https://github.com/milvus-io/milvus/releases/download/v2.6.0/milvus-standalone-docker-compose-gpu.yml -O docker-compose.yml
```

Bring Milvus up:

```bash
docker compose up -d
docker compose ps
```

Tail logs to confirm full Milvus startup (not just `tini`):

```bash
docker logs -f milvus-standalone
```

WebUI (optional):
```
http://127.0.0.1:9091/webui/
```

> If you only see `tini` repeatedly, you’re using the wrong compose file. Use the official one above.



## Step 7 — Python client quick test (`pymilvus`)

Install the client in your venv:

```bash
pip install pymilvus
```

Connect and print server version:

```python
from pymilvus import connections, utility
connections.connect(host="127.0.0.1", port="19530")
print("Connected to Milvus:", utility.get_server_version())
```

If you see a version string, your GPU Milvus instance is running and reachable 🎉



## Troubleshooting

- **NO_PUBKEY DDCAE044F796ECB0** → Re-run Step 2 exactly (keyring + signed-by).
- **Unit docker.service not found** in Ubuntu → Normal on Docker Desktop. Restart via Docker Desktop or `wsl --shutdown`.
- **Default Runtime shows runc** → Normal; GPU still works. Set default in Docker Desktop’s Engine JSON if desired.
- **nvidia-smi fails inside container** → Update Windows NVIDIA driver, ensure Docker Desktop → WSL Integration is enabled, re-run Step 5.
